In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
# Thiết lập tùy chọn hiển thị của pandas
pd.set_option('display.max_columns', None)  # Hiển thị tất cả các cột
pd.set_option('display.max_rows', None)     # Hiển thị tất cả các hàng

# Đặt lại tùy chọn hiển thị về giá trị mặc định nếu cần
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_rows')

In [3]:
def check_missing_data(df, columns):
    """
    Kiểm tra dữ liệu bị thiếu trong các cột được chỉ định của DataFrame.
    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.
        columns (list): Danh sách tên các cột cần kiểm tra.
    Returns:
        DataFrame: DataFrame chứa các dòng dữ liệu có dữ liệu bị thiếu trong các cột được chỉ định.
    """
    missing_data = df[df[columns].isnull().any(axis=1)]
    if missing_data.empty:
        print("Không có dữ liệu bị thiếu trong các cột:", ", ".join(columns))
    else:
        print("Số lượng dòng dữ liệu bị thiếu trong các cột", ", ".join(columns), "là:", missing_data.shape[0])
        print("Các dòng dữ liệu bị thiếu trong các cột", ", ".join(columns), "là:")
        # print(missing_data)
    return missing_data

def check_duplicate_data(df):
    """
    Hàm để kiểm tra và theo dõi dữ liệu trùng lặp trong DataFrame.

    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.

    Returns:
        DataFrame: DataFrame chứa các dòng dữ liệu trùng lặp.
    """
    # Kiểm tra dữ liệu trùng lặp
    duplicate_data = df[df.duplicated()]

    # In thông báo nếu không có dữ liệu trùng lặp
    if duplicate_data.empty:
        print("Không có dữ liệu trùng lặp.")
    else:
      print("Những dữ liệu trùng lặp: ", duplicate_data)
    return duplicate_data


def count_date_formats(df, column, print_result=False):
    """
    Hàm để kiểm tra và theo dõi số lượng định dạng của dữ liệu ngày tháng trong một cột.

    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.
        column (str): Tên của cột cần kiểm tra.
        print_result (bool, optional): True nếu muốn in kết quả ra màn hình, False nếu không (mặc định là False).

    Returns:
        dict: Một từ điển với các định dạng ngày tháng làm khóa và số lượng dòng có mỗi định dạng làm giá trị.
    """
    # Chuyển đổi các giá trị của cột thành chuỗi định dạng ngày tháng
    date_strings = df[column].apply(lambda x: x.strftime('%d/%m/%Y') if isinstance(x, pd.Timestamp) else x)

    # Đếm số lượng giá trị duy nhất cho mỗi định dạng ngày tháng
    format_counts = date_strings.value_counts()

    # Chuyển đổi kết quả thành từ điển
    format_counts_dict = format_counts.to_dict()

    # In kết quả nếu cần
    if print_result:
        print("Số lượng định dạng của dữ liệu ngày tháng:")
        for format, count in format_counts_dict.items():
            print(f"{format}: {count} dòng")

    return format_counts_dict

def score_labeling(df, column_name, bins=[0, 4, 5, 6, 7, 8, 9, 10], labels=['kém', 'yếu', 'trung bình', 'trung bình khá', 'khá', 'giỏi', 'xuất sắc']):
    """
    Hàm để chuyển điểm thành biến phân loại kết quả học tập và trả về cột mới.

    Parameters:
        df (DataFrame): DataFrame chứa dữ liệu.
        column_name (str): Tên cột chứa điểm cần phân loại.
        bins (list): Danh sách khoảng giá trị cho biến phân loại. Mặc định là [0, 4, 5, 6, 7, 8, 9, 10].
        labels (list): Danh sách nhãn cho các khoảng giá trị. Mặc định là ['kém', 'yếu', 'trung bình', 'trung bình khá', 'khá', 'giỏi', 'xuất sắc'].

    Returns:
        Series: Cột mới chứa biến phân loại.
    """
    # Thêm cột mới vào DataFrame chứa biến phân loại
    df['{}_xl'.format(column_name)] = pd.cut(df[column_name], bins=bins, labels=labels, right=False)
    return df['{}_xl'.format(column_name)]

## Data Quality Rules


In [24]:
# Hàm đánh giá tính duy nhất (Uniqueness) cho toàn bộ dòng dữ liệu
def check_uniqueness(df):
    duplicate_count = df.duplicated().sum()
    total_count = len(df)
    return (total_count - duplicate_count) / total_count

# Hàm đánh giá tính kịp thời (Timeliness)
def check_timeliness(row, current_year):
    try:
        # Giả sử dữ liệu cập nhật mỗi năm
        if row['namhoc'] <= current_year:
            return 1
        else:
            return 0
    except:
        return 0

# Hàm đánh giá tính nhất quán (Consistency)
def check_consistency(row):
    try:
        # Kiểm tra điểm trong khoảng 0-10
        if 0 <= row['diem_hp'] <= 10:
            return 1
        else:
            return 0
    except:
        return 0
    
# Hàm đánh giá tính đầy đủ (Completeness)
def check_completeness(row):
    required_columns = ['masv', 'mamh', 'hocky','namhoc', 'diem_hp']
    missing_count = row[required_columns].isna().sum()
    return 1 if missing_count == 0 else 0

# Hàm đánh giá tính hợp lệ (Validity)
def check_validity(row):
    try:
        # Kiểm tra điểm trong khoảng 0-10
        if 0 <= row['diem_hp'] <= 10:
            return 1
        else:
            return 0
    except:
        return 0

## CONNECT TO DATASET: DIEM_THU.XLSX

In [4]:
# Loading the New Customer Data from the excel file
df = pd.read_excel('data/diem_Thu.xlsx')

In [5]:
# Checking first 5 records from Data
df.head(5)

,masv,mamh,malop,sotc,magv,hocky,namhoc,diem_qt,diem_th,diem_gk,diem_ck,diem_hp,trangthai,tinhtrang,mamh_tt
0,6520001,CARC1,K1C1_2,3,NaN,1,2006,NaN,NaN,NaN,NaN,2.0,2,0,NaN
1,6520001,CSC21,K1C1_2,5,NaN,1,2006,NaN,NaN,NaN,NaN,5.0,1,0,NaN
2,6520001,ENG02,K1C1_2,0,NaN,1,2006,NaN,NaN,NaN,NaN,5.5,1,0,NaN
3,6520001,MAT21,K1C1_2,4,NaN,1,2006,NaN,NaN,NaN,NaN,6.5,1,0,NaN
4,6520001,PEDU1,K1C1_2,0,NaN,1,2006,NaN,NaN,NaN,NaN,7.0,1,0,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674273 entries, 0 to 674272
Data columns (total 15 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   masv       674273 non-null  int64  
 1   mamh       674273 non-null  object 
 2   malop      674258 non-null  object 
 3   sotc       674273 non-null  int64  
 4   magv       485337 non-null  object 
 5   hocky      674273 non-null  int64  
 6   namhoc     674273 non-null  int64  
 7   diem_qt    297601 non-null  float64
 8   diem_th    238559 non-null  float64
 9   diem_gk    303975 non-null  float64
 10  diem_ck    513427 non-null  float64
 11  diem_hp    673659 non-null  float64
 12  trangthai  674273 non-null  int64  
 13  tinhtrang  674273 non-null  int64  
 14  mamh_tt    2741 non-null    object 
dtypes: float64(5), int64(6), object(4)
memory usage: 77.2+ MB


Total Records

In [7]:
print("Total records (rows) in the dataset : {}".format(df.shape[0]))
print("Total columns (features) in the dataset : {}".format(df.shape[1]))

Total records (rows) in the dataset : 674273
Total columns (features) in the dataset : 15


# EXPLORE DATA ANALYTICS

1 Missing Values Check:

In [8]:
df.isnull().sum()

masv              0
mamh              0
malop            15
sotc              0
magv         188936
hocky             0
namhoc            0
diem_qt      376672
diem_th      435714
diem_gk      370298
diem_ck      160846
diem_hp         614
trangthai         0
tinhtrang         0
mamh_tt      671532
dtype: int64

In [9]:
# Percentage of missing values
df.isnull().mean()*100

masv          0.000000
mamh          0.000000
malop         0.002225
sotc          0.000000
magv         28.020698
hocky         0.000000
namhoc        0.000000
diem_qt      55.863426
diem_th      64.619820
diem_gk      54.918112
diem_ck      23.854729
diem_hp       0.091061
trangthai     0.000000
tinhtrang     0.000000
mamh_tt      99.593488
dtype: float64

1.1 malop

In [10]:
# Phân tích thống kê
summary_stats = df.describe()
missing_values = df.isnull().sum()

In [11]:
df.mamh_tt.unique()

array([nan, 'NT113', 'CE115', 'IS202', 'CS108', 'SE104', 'CS109', 'ENG13',
       'ENG11', 'PHIL1', 'STAT11', 'MAT11', 'OOPT1', 'DSAL1', 'NT304',
       'PH001', 'MAT02', 'ENG01', 'SS001', 'ENG12', 'CSC12', 'PEDU2',
       'IT003', 'MA003', 'ENG03', 'CS104', 'CS102', 'CNET1', 'CS101',
       'CS322', 'CS103', 'NT101', 'CARC1', 'CSC01', 'DBSS1', 'IS204',
       'MAT04', 'LIA01', 'MAT01', 'STA01', 'ENG02', 'LIA11', 'PEDU1',
       'OSYS1', 'WINP1', 'ITEW1', 'IS103', 'IS102', 'ENG15', 'IS4263',
       'VCPL1', 'HCMT1', 'IT008', 'NT110', 'PHY01', 'IS104', 'IS206',
       'CE102', 'PHY02', 'IS501', 'CE202', 'SE212', 'SE341', 'SE311',
       'SE213', 'IS107', 'IS311', 'IS106', 'SE210', 'CS217', 'CE106',
       'CE109', 'PE001', 'SE103', 'NT301', 'IS205', 'SE209', 'NT202',
       'NT201', 'SE207', 'CE113', 'NT102', 'CE302', 'CE110', 'NT305',
       'SE322', 'NT321', 'NT302', 'SE321', 'SE211', 'SE208', 'NT203',
       'SE417', 'SE418', 'NT108', 'CE116', 'CE112', 'SS005', 'EN005',
       'NT107

In [12]:
# Tính tỷ lệ dữ liệu thiếu của toàn bộ DataFrame
missing_data_ratio = df.isna().sum().sum() / (df.shape[0] * df.shape[1]) * 100
print("Tỉ lệ dữ liệu thiếu của toàn bộ DataFrame: {:.2f}%".format(missing_data_ratio))
completeness_score = 100 - missing_data_ratio
print("Completeness: {:.2f}%".format(100 - missing_data_ratio))

Tỉ lệ dữ liệu thiếu của toàn bộ DataFrame: 21.80%
Completeness: 78.20%


In [13]:
# Tạo DataFrame với các dimension và điểm tương ứng
data = {
  'Total Rows': [df.shape[0]],
  'Total Features': [df.shape[1]],
  'Completeness': [completeness_score],
  'Consitency': [90],
  'Validity': [90],
  'Uniqueness': [90]
}

# Tạo DataFrame từ dữ liệu
df_scores = pd.DataFrame(data)

In [14]:
df_scores.to_csv('report/diem_DQ.csv', index = False)

## Data Quality Assessment

In [20]:
# current_year là năm hiện tại
current_year = 2023

# Áp dụng các hàm đánh giá cho từng dòng
df['completeness'] = df.apply(check_completeness, axis=1)
df['consistency'] = df.apply(check_consistency, axis=1)
df['timeliness'] = df.apply(check_timeliness, axis=1, current_year=current_year)
df['validity'] = df.apply(check_validity, axis=1)

In [25]:
# Tính điểm trung bình cho từng dimension
completeness_score = df['completeness'].mean()
consistency_score = df['consistency'].mean()
timeliness_score = df['timeliness'].mean()
validity_score = df['validity'].mean()
uniqueness_score = check_uniqueness(df)

In [31]:
# Lưu kết quả vào DataFrame mới
# quality_scores = pd.DataFrame({
#     'dimension': ['completeness', 'consistency', 'timeliness', 'validity', 'uniqueness'],
#     'score': [completeness_score, consistency_score, timeliness_score, validity_score, uniqueness_score]
# })

quality_scores = pd.DataFrame({
  'Total Rows': [df.shape[0]],
  'Total Features': [df.shape[1]],
  'Completeness': [completeness_score],
  'Consitency': [consistency_score],
  'Validity': [validity_score],
  'Uniqueness': [uniqueness_score],
  'Timeliness': [timeliness_score]
})

print(quality_scores)

   Total Rows  Total Features  Completeness  Consitency  Validity  Uniqueness  \
0      674273              19      0.999089     0.99908   0.99908         1.0   

   Timeliness  
0         1.0  


In [32]:
# Lưu kết quả vào file CSV
quality_scores.to_csv('report/diem_DQ.csv', index=False)